In [652]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.options.mode.chained_assignment = None

Function to fix the columns names

In [653]:
def fix_column_names(df):
    for index, column in enumerate(df.columns):
        if '/' in column:
            names = column.split('/')
            first = names[0].strip()
            second = names[0].strip() + " " + names[1].strip()
            
            df.rename(columns={column: first}, inplace=True)
            df.rename(columns={df.columns[index + 1]: second}, inplace=True)
    
    return df

Function to normalize the data in 90 minutes.

In [651]:
def norm_90(df, cols):
    # Sum of given columns
    df_90 = df[cols].sum(numeric_only=True)

    # Create a column for 90m played
    df_90["90s"] = int(df_90["Minutes played"]) / 90

    df_90 = pd.DataFrame(df_90).T

    columns_to_divide = cols[2:]

    # Divide these columns by the values in the '90s' column
    df_90[columns_to_divide] = df_90[columns_to_divide].div(df_90['90s'], axis=0)

    # Drop the 90s column
    df_90.drop("90s", axis=1, inplace=True)

    # Rename the columns to per 90'
    for i in range(1, len(df_90.columns)):
        df_90.columns.values[i] = df_90.columns[i] + '/90'

    df_90["Team"] = df["Team"].iloc[0]
    
    return df_90

Concat all data and per 90 metrics

In [654]:
def team_data(df_list, cols):
    temp_list = []

    for df in df_list:
        df = norm_90(df, cols)
        temp_list.append(df)
    
    teams_df = pd.concat(temp_list)
    teams_df = teams_df.set_index("Team").rename_axis(None)

    return teams_df
     

Import Cremaschi data into dataframe

In [655]:
cremaschi_df = pd.read_excel("USA Data/Player stats B. Cremaschi.xlsx", )

cremaschi_df.head(3)

,Match,Competition,Date,Position,Minutes played,Total actions / successful,Unnamed: 6,Goals,Assists,Shots / on target,Unnamed: 10,xG,Passes / accurate,Unnamed: 13,Long passes / accurate,Unnamed: 15,Crosses / accurate,Unnamed: 17,Dribbles / successful,Unnamed: 19,Duels / won,Unnamed: 21,Aerial duels / won,Unnamed: 23,Interceptions,Losses / own half,Unnamed: 26,Recoveries / opp. half,Unnamed: 28,Yellow card,Red card,Defensive duels / won,Unnamed: 32,Loose ball duels / won,Unnamed: 34,Sliding tackles / successful,Unnamed: 36,Clearances,Fouls,Yellow cards,Red cards,Shot assists,Offensive duels / won,Unnamed: 43,Touches in penalty area,Offsides,Progressive runs,Fouls suffered,Through passes / accurate,Unnamed: 49,xA,Second assists,Passes to final third / accurate,Unnamed: 53,Passes to penalty area / accurate,Unnamed: 55,Received passes,Forward passes / accurate,Unnamed: 58,Back passes / accurate,Unnamed: 60
0,Nashville SC - Inter Miami 1:2,United States. MLS,2024-06-30,"LCMF3, RW",99,73,46,0,0,2,0,0.33,43,39,0,0,1,1,3,1,16,6,1,1,0,8,2,6,2,0,0,3,2,2,0,0,0,0,0,0,0,2,10,3,3,1,1,0,0,0,0.02,0,3,3,2,2,31,12,10,14,13
1,Inter Miami - Columbus Crew 2:1,United States. MLS,2024-06-20,"RCMF, RW",98,63,42,0,0,1,0,0.02,42,35,2,1,1,1,0,0,15,7,1,0,5,13,8,9,2,0,0,5,4,2,1,1,1,0,0,0,0,1,7,2,1,0,0,0,0,0,0.10,0,5,3,1,1,27,12,9,7,7
2,Philadelphia Union - Inter Miami 1:2,United States. MLS,2024-06-16,"LCMF, LCMF3",98,77,45,0,1,3,0,0.31,43,36,5,3,0,0,1,0,23,9,4,2,3,12,4,4,3,0,0,2,2,3,2,0,0,2,0,0,0,1,14,3,4,0,2,0,0,0,0.28,0,7,5,2,2,35,12,10,11,10


Clean the column names

In [656]:
cremaschi_df = fix_column_names(cremaschi_df)

We want to create two different df, one for the academy and the other for the first team

In [657]:
cremaschi_academy_df = cremaschi_df[cremaschi_df["Match"].str.contains("Inter Miami II", na=False)]
cremaschi_academy_df["Team"] = "Inter Miami II"
#cremaschi_academy_df.reset_index(inplace=True, drop=True)

cremaschi_firstteam_df = cremaschi_df[~cremaschi_df['Match'].str.contains('Inter Miami II', na=False) & ~cremaschi_df['Match'].str.contains('United States', na=False)]
cremaschi_firstteam_df["Team"] = "Inter Miami"
#cremaschi_firstteam_df.reset_index(inplace=True, drop=True)

cremaschi_nationalteam_df = cremaschi_df[cremaschi_df["Match"].str.contains("United States", na=False)]
cremaschi_nationalteam_df["Team"] = "United States U20"
#cremaschi_nationalteam_df.reset_index(inplace=True, drop=True)

Calculate metrics to 90 minutes

In [658]:
cols = ["Team", 'Minutes played',
        'Total actions', 'Total actions successful', 'Goals', 'Assists',
        'Shots', 'Shots on target', 'xG', 'Passes', 'Passes accurate',
        'Long passes', 'Long passes accurate', 'Crosses', 'Crosses accurate',
        'Dribbles', 'Dribbles successful', 'Duels', 'Duels won', 'Aerial duels',
        'Aerial duels won', 'Interceptions', 'Losses', 'Losses own half',
        'Recoveries', 'Recoveries opp. half', 'Yellow card', 'Red card',
        'Defensive duels', 'Defensive duels won', 'Loose ball duels',
        'Loose ball duels won', 'Sliding tackles', 'Sliding tackles successful',
        'Clearances', 'Fouls', 'Yellow cards', 'Red cards', 'Shot assists',
        'Offensive duels', 'Offensive duels won', 'Touches in penalty area',
        'Offsides', 'Progressive runs', 'Fouls suffered', 'Through passes',
        'Through passes accurate', 'xA', 'Second assists',
        'Passes to final third', 'Passes to final third accurate',
        'Passes to penalty area', 'Passes to penalty area accurate',
        'Received passes', 'Forward passes', 'Forward passes accurate',
        'Back passes', 'Back passes accurate']

all_df = [cremaschi_academy_df, cremaschi_firstteam_df, cremaschi_nationalteam_df]
cremashi_teams_df = team_data(all_df, cols)
cremashi_teams_df

,Minutes played,Total actions/90,Total actions successful/90,Goals/90,Assists/90,Shots/90,Shots on target/90,xG/90,Passes/90,Passes accurate/90,Long passes/90,Long passes accurate/90,Crosses/90,Crosses accurate/90,Dribbles/90,Dribbles successful/90,Duels/90,Duels won/90,Aerial duels/90,Aerial duels won/90,Interceptions/90,Losses/90,Losses own half/90,Recoveries/90,Recoveries opp. half/90,Yellow card/90,Red card/90,Defensive duels/90,Defensive duels won/90,Loose ball duels/90,Loose ball duels won/90,Sliding tackles/90,Sliding tackles successful/90,Clearances/90,Fouls/90,Yellow cards/90,Red cards/90,Shot assists/90,Offensive duels/90,Offensive duels won/90,Touches in penalty area/90,Offsides/90,Progressive runs/90,Fouls suffered/90,Through passes/90,Through passes accurate/90,xA/90,Second assists/90,Passes to final third/90,Passes to final third accurate/90,Passes to penalty area/90,Passes to penalty area accurate/90,Received passes/90,Forward passes/90,Forward passes accurate/90,Back passes/90,Back passes accurate/90
Inter Miami II,1181.0,68.204911,40.160881,0.381033,0.152413,1.066892,0.457240,0.087638,32.692633,26.977138,1.676545,0.990686,0.685859,0.304826,3.048264,1.676545,23.471634,11.354784,1.905165,0.990686,2.514818,9.602032,4.115157,6.553768,2.133785,9.068586,0.0,10.592718,6.096528,2.819644,1.066892,0.22862,0.152413,0.762066,1.600339,0.304826,0.0,0.914479,8.154107,3.200677,1.371719,0.00000,1.524132,1.295512,0.609653,0.152413,0.117358,0.152413,4.115157,2.895851,1.295512,0.533446,23.624047,10.821338,7.773074,5.563082,4.877223
Inter Miami,3366.0,72.219251,46.149733,0.133690,0.133690,1.096257,0.347594,0.139305,40.320856,35.374332,1.283422,0.668449,0.748663,0.240642,2.219251,0.935829,21.764706,9.973262,2.272727,0.962567,2.700535,9.064171,3.716578,5.695187,2.513369,9.064171,0.0,8.048128,5.187166,2.566845,1.096257,0.26738,0.160428,0.695187,1.069519,0.187166,0.0,0.695187,8.877005,2.727273,1.978610,0.13369,1.203209,1.016043,0.695187,0.374332,0.103209,0.026738,3.903743,3.074866,1.497326,0.935829,31.577540,9.331551,7.192513,9.358289,8.796791
United States U20,233.0,86.909871,56.008584,0.386266,0.000000,1.931330,0.772532,0.312876,48.283262,41.330472,0.386266,0.000000,2.703863,1.158798,6.180258,3.090129,26.652361,12.360515,1.931330,1.158798,6.566524,11.974249,4.635193,11.587983,8.111588,0.000000,0.0,8.497854,4.248927,3.862661,2.703863,0.00000,0.000000,0.000000,0.386266,0.000000,0.0,1.931330,12.360515,4.248927,4.248927,0.00000,1.158798,1.545064,1.158798,0.386266,0.262661,0.386266,6.566524,5.021459,3.476395,1.931330,35.922747,15.064378,11.587983,11.201717,11.201717


Import Ricardo Pepi data into dataframe

In [659]:
pepi_df = pd.read_excel('USA Data/Player stats R. Pepi.xlsx')

fix column names

In [660]:
pepi_df = fix_column_names(pepi_df)

In [661]:
pepi_u17_df = pepi_df[pepi_df["Match"].str.contains("United States U17", na=False)]
pepi_u17_df["Team"] = "United States U17"

pepi_PSV_df = pepi_df[pepi_df["Match"].str.contains("PSV", na=False)]
pepi_PSV_df["Team"] = "PSV"

pepi_USA_df = pepi_df[pepi_df['Match'].str.contains('United States', na=False) & ~pepi_df['Match'].str.contains('U17', na=False)  & ~pepi_df['Match'].str.contains('U18', na=False)]
pepi_USA_df["Team"] = "United States"

pepi_groningen_df = pepi_df[pepi_df["Match"].str.contains("Groningen", na=False)]
pepi_groningen_df["Team"] = "Groningen"

pepi_northtexas_df = pepi_df[pepi_df["Match"].str.contains("North Texas", na=False)]
pepi_northtexas_df["Team"] = "North Texas"

pepi_dallas_df = pepi_df[pepi_df["Match"].str.contains("Dallas", na=False)]
pepi_dallas_df["Team"] = "Dallas"

normalize data to 90 min

In [662]:
cols = ["Team", 'Minutes played',
        'Total actions', 'Total actions successful', 'Goals', 'Assists',
        'Shots', 'Shots on target', 'xG', 'Passes', 'Passes accurate',
        'Long passes', 'Long passes accurate', 'Crosses', 'Crosses accurate',
        'Dribbles', 'Dribbles successful', 'Duels', 'Duels won', 'Aerial duels',
        'Aerial duels won', 'Interceptions', 'Losses', 'Losses own half',
        'Recoveries', 'Recoveries opp. half', 'Yellow card', 'Red card',
        'Defensive duels', 'Defensive duels won', 'Loose ball duels',
        'Loose ball duels won', 'Sliding tackles', 'Sliding tackles successful',
        'Clearances', 'Fouls', 'Yellow cards', 'Red cards', 'Shot assists',
        'Offensive duels', 'Offensive duels won', 'Touches in penalty area',
        'Offsides', 'Progressive runs', 'Fouls suffered', 'Through passes',
        'Through passes accurate', 'xA', 'Second assists',
        'Passes to final third', 'Passes to final third accurate',
        'Passes to penalty area', 'Passes to penalty area accurate',
        'Received passes', 'Forward passes', 'Forward passes accurate',
        'Back passes', 'Back passes accurate']

all_df = [pepi_u17_df, pepi_dallas_df, pepi_groningen_df, pepi_USA_df, pepi_PSV_df, pepi_northtexas_df]
pepi_teams_df = team_data(all_df, cols)
pepi_teams_df


,Minutes played,Total actions/90,Total actions successful/90,Goals/90,Assists/90,Shots/90,Shots on target/90,xG/90,Passes/90,Passes accurate/90,Long passes/90,Long passes accurate/90,Crosses/90,Crosses accurate/90,Dribbles/90,Dribbles successful/90,Duels/90,Duels won/90,Aerial duels/90,Aerial duels won/90,Interceptions/90,Losses/90,Losses own half/90,Recoveries/90,Recoveries opp. half/90,Yellow card/90,Red card/90,Defensive duels/90,Defensive duels won/90,Loose ball duels/90,Loose ball duels won/90,Sliding tackles/90,Sliding tackles successful/90,Clearances/90,Fouls/90,Yellow cards/90,Red cards/90,Shot assists/90,Offensive duels/90,Offensive duels won/90,Touches in penalty area/90,Offsides/90,Progressive runs/90,Fouls suffered/90,Through passes/90,Through passes accurate/90,xA/90,Second assists/90,Passes to final third/90,Passes to final third accurate/90,Passes to penalty area/90,Passes to penalty area accurate/90,Received passes/90,Forward passes/90,Forward passes accurate/90,Back passes/90,Back passes accurate/90
United States U17,790.0,57.759494,23.582278,0.569620,0.113924,2.164557,1.367089,0.480759,18.911392,14.126582,0.227848,0.113924,0.569620,0.113924,2.050633,0.911392,22.101266,7.746835,3.645570,0.797468,1.936709,11.164557,1.594937,3.531646,2.278481,0.000000,0.0,6.379747,3.075949,3.531646,1.025316,0.000000,0.000000,0.227848,1.708861,0.000000,0.0,0.797468,8.544304,2.848101,4.329114,0.569620,0.455696,0.569620,0.569620,0.227848,0.060380,0.000000,1.481013,1.025316,0.911392,0.683544,11.848101,3.189873,2.164557,5.696203,4.784810
Dallas,3110.0,52.379421,23.064309,0.463023,0.086817,2.372990,1.186495,0.384309,19.475884,15.453376,0.463023,0.173633,0.347267,0.086817,1.765273,0.723473,17.276527,5.932476,3.183280,0.868167,1.620579,8.392283,2.749196,2.864952,1.794212,2.430868,0.0,4.745981,2.430868,3.096463,0.897106,0.231511,0.144695,0.491961,0.897106,0.028939,0.0,0.897106,6.250804,1.736334,3.675241,0.434084,0.520900,0.723473,0.665595,0.289389,0.079003,0.086817,1.186495,0.752412,0.810289,0.434084,15.163987,3.038585,1.996785,6.077170,5.440514
Groningen,2724.0,53.359031,21.872247,0.495595,0.066079,1.916300,0.892070,0.313877,16.552863,13.281938,0.231278,0.198238,0.231278,0.099119,1.651982,0.726872,21.211454,7.202643,6.409692,1.982379,2.610132,10.308370,2.940529,2.841410,1.751101,1.949339,0.0,3.898678,2.378855,4.262115,1.288546,0.033040,0.033040,1.057269,0.627753,0.066079,0.0,0.594714,6.640969,1.552863,3.634361,0.495595,0.528634,0.198238,0.132159,0.033040,0.062775,0.066079,0.825991,0.495595,0.660793,0.429515,11.464758,1.982379,1.123348,5.748899,5.088106
United States,1346.0,48.945022,21.597325,0.601783,0.066865,2.741456,1.471025,0.578380,16.916790,14.576523,0.066865,0.066865,0.200594,0.133730,1.337296,0.401189,16.783061,5.215453,2.875186,0.601783,1.939079,8.023774,1.738484,3.142645,2.540862,0.000000,0.0,3.677563,2.139673,4.346211,1.671620,0.200594,0.133730,0.267459,0.735513,0.000000,0.0,0.468053,5.884101,0.802377,4.413076,0.267459,0.668648,0.200594,0.200594,0.066865,0.039450,0.000000,1.069837,0.668648,0.401189,0.200594,12.303120,2.607727,1.805349,5.817236,5.549777
PSV,1159.0,53.192407,24.305436,0.776531,0.232959,2.640207,1.397757,0.587058,21.199310,16.695427,0.698878,0.388266,0.232959,0.077653,1.320104,0.621225,19.102675,5.823986,3.805004,1.009491,1.708369,9.163072,3.028473,2.407248,1.553063,19.646247,0.0,3.882657,2.329594,3.805004,1.087144,0.155306,0.077653,0.776531,0.776531,0.232959,0.0,1.009491,7.610009,1.397757,4.969802,0.543572,0.776531,0.310613,0.232959,0.077653,0.072994,0.000000,1.320104,0.854185,1.009491,0.621225,15.763589,3.572045,2.407248,6.600518,5.746333
North Texas,1279.0,52.212666,24.910086,0.774042,0.000000,4.010946,1.618452,0.761376,21.884285,16.958561,0.774042,0.562940,0.774042,0.211102,2.673964,1.266615,15.903049,5.981235,2.814699,1.196247,0.703675,8.936669,1.477717,2.251759,1.688819,0.000000,0.0,3.588741,2.040657,1.970289,0.703675,0.140735,0.070367,0.140735,0.281470,0.000000,0.0,0.422205,7.529320,2.040657,3.799844,0.84